## Q1. Running Elastic

**Q:** What's the version.build_hash value?

**A:** build_hash : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [1]:
!curl localhost:9200

{
  "name" : "b3ea15835a88",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Ld_rtEXeSSyOPUW3CrOHNg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Q2. Indexing the data
**Q:** Which function do you use for adding your data to elastic?

**A:** index


## Q3. Searching


**Q:** What's the score for the top ranking result?

**A:** top score : 84.050095


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████| 948/948 [00:25<00:00, 37.20it/s]


In [48]:
for index, result in enumerate(response['hits']['hits']):
    print(index + 1, '|' , result['_score'])


1 | 84.050095
2 | 51.04628
3 | 49.938507
4 | 45.275463
5 | 45.255775


In [14]:
query = 'How do I execute a command in a running docker container?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            }
                }
            }
        }
    


response = es_client.search(index=index_name, body= search_query)

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

**Q:** Return 3 results. What's the 3rd question returned by the search engine?

**A:** How do I copy files from a different folder into docker container’s working directory?

In [49]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body= search_query)


In [50]:
for index, hit in enumerate(response['hits']['hits']):
    print(index + 1, "|" ,hit['_source']['question'])

1 | How do I debug a docker container?
2 | How do I copy files from my local machine to docker container?
3 | How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt
**Q:** What's the length of the resulting prompt? (use the len function)

**A:** 1462/1424


In [59]:
# create modular functions that can be used to build prompt
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body= search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, search_results):
    context = ""
    context_template = """
    
    Q: {question}
    A: {text}
    """.strip()
    
    for hit in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [60]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)

len(prompt)

1424

## Q6. Tokens

**Q:** Use the encode function. How many tokens does our prompt have?

**A:** 322/306

In [61]:
import os
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

len(encoding.encode(str(prompt)))

306